#### Different Propirtoin of of Retain/Forget


The TOFU benchmark has 3 different proportions/percentages : 1,5,10 for their forget sets. All of my work I am currently doing on the 10 percent setting. I want my code to expand, such that I have 1,5, 25 and 50 settings as well.

In [29]:
import json
import pandas as pd


forget_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10.json'
retain_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90.json'
full_data_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full2.json'


with open(forget_path, 'r') as f:
    forget_data = json.load(f)
with open(retain_path, 'r') as f:
    retain_data = json.load(f)
with open(full_data_path, 'r') as f:
    full_data = json.load(f)


forget10_df = pd.DataFrame(forget_data)
retain90_df = pd.DataFrame(retain_data)
full_df = pd.DataFrame(full_data)


In [8]:
full_df.loc[full_df['question'].isin(forget10_df['question']),'config10'] = 'forget'
full_df.loc[full_df['question'].isin(retain90_df['question']),'config10'] = 'retain'

#### Create sets with 50,25,25 and 1 percent. 
Make sure they are overlapping (i.e all persons from the 50 set are present in the 25,10,5,1 and so on downstream).

In [41]:
import pandas as pd
import numpy as np

unique_names = retain90_df['subject'].unique()
count = 80
selected_names = np.random.choice(unique_names, size=count, replace=False)
new50_df = retain90_df[retain90_df['subject'].isin(selected_names)]
forget50_df = pd.concat([forget10_df, new50_df], ignore_index=True)


In [45]:
import pandas as pd
import numpy as np

unique_names = new50_df['subject'].unique()
count = 30
selected_names = np.random.choice(unique_names, size=count, replace=False)
new15_df = retain90_df[retain90_df['subject'].isin(selected_names)]
forget25_df = pd.concat([forget10_df, new15_df], ignore_index=True)

In [ ]:
import pandas as pd
import numpy as np

unique_names = forget10_df['subject'].unique()
count = 10
selected_names = np.random.choice(unique_names, size=count, replace=False)
forget5_df = forget10_df[forget10_df['subject'].isin(selected_names)]

In [51]:
import pandas as pd
import numpy as np

unique_names = forget5_df['subject'].unique()
count = 2
selected_names = np.random.choice(unique_names, size=count, replace=False)
forget1_df = forget5_df[forget5_df['subject'].isin(selected_names)]

In [68]:
retain_forget_df = pd.concat([forget10_df, retain90_df], ignore_index=True)

In [69]:
retain50_df = retain90_df[~retain90_df['subject'].isin(new50_df['subject'])]
retain75_df = retain90_df[~retain90_df['subject'].isin(new15_df['subject'])]
retain95_df = retain_forget_df[~retain_forget_df['subject'].isin(forget5_df['subject'])]
retain99_df = retain_forget_df[~retain_forget_df['subject'].isin(forget1_df['subject'])]

In [81]:
print(forget1_df['subject'].isin(retain99_df['subject']).any())
print(forget5_df['subject'].isin(retain95_df['subject']).any())
print(forget10_df['subject'].isin(retain90_df['subject']).any())
print(forget25_df['subject'].isin(retain75_df['subject']).any())
print(forget50_df['subject'].isin(retain50_df['subject']).any())

False
False
False
False
False


In [95]:
forget1_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget1.json', orient='records', lines=False)
forget5_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget5.json', orient='records', lines=False)
forget25_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget25.json', orient='records', lines=False)   
forget50_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget50.json', orient='records', lines=False)

retain50_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain50.json', orient='records', lines=False)
retain75_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain75.json', orient='records', lines=False)
retain95_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain95.json', orient='records', lines=False)
retain99_df.to_json('/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain99.json', orient='records', lines=False)

In [88]:
split_names10_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/split_person_names/forget10_names.json'

with open(split_names10_path, 'r') as f:
    split_names = json.load(f)

##### Save split names for Evaluation Metrics

In [94]:
import json

# Your existing name lists - now with unique names only
split_forgetnames1 = forget1_df['subject'].unique().tolist()
split_forgetnames5 = forget5_df['subject'].unique().tolist()
split_forgetnames25 = forget25_df['subject'].unique().tolist()
split_forgetnames50 = forget50_df['subject'].unique().tolist()

split_retainnames50 = retain50_df['subject'].unique().tolist()
split_retainnames75 = retain75_df['subject'].unique().tolist()
split_retainnames95 = retain95_df['subject'].unique().tolist()
split_retainnames99 = retain99_df['subject'].unique().tolist()

# Create a mapping for easier access
forget_names_dict = {
    1: split_forgetnames1,
    5: split_forgetnames5,
    25: split_forgetnames25,
    50: split_forgetnames50
}

retain_names_dict = {
    50: split_retainnames50,
    75: split_retainnames75,
    95: split_retainnames95,
    99: split_retainnames99
}

# Save forget and retain files
for p in [1, 5, 25, 50]:
    forget_p = p
    retain_p = 100 - p
    
    # Save forget names
    split_forget_path = f'/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/split_person_names/forget{forget_p}_names.json'
    with open(split_forget_path, 'w') as f:
        json.dump(forget_names_dict[forget_p], f, indent=2)
    
    # Save retain names
    split_retain_path = f'/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/split_person_names/retain{retain_p}_names.json'
    with open(split_retain_path, 'w') as f:
        json.dump(retain_names_dict[retain_p], f, indent=2)
    
    print(f"Saved forget{forget_p}_names.json ({len(forget_names_dict[forget_p])} unique names) and retain{retain_p}_names.json ({len(retain_names_dict[retain_p])} unique names)")

Saved forget1_names.json (2 unique names) and retain99_names.json (198 unique names)
Saved forget5_names.json (10 unique names) and retain95_names.json (190 unique names)
Saved forget25_names.json (50 unique names) and retain75_names.json (150 unique names)
Saved forget50_names.json (100 unique names) and retain50_names.json (100 unique names)
